In [1]:
import pickle
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


In [2]:
from sklearn.pipeline import make_pipeline

In [4]:
pip install mlflow

  Using cached mlflow-2.21.0-py3-none-any.whl.metadata (30 kB)
  Using cached mlflow_skinny-2.21.0-py3-none-any.whl.metadata (31 kB)
  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached alembic-1.15.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached sqlalchemy-2.0.39-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached databricks_sdk-0.46.0-py3-none-any.whl.metadata (38 kB)
  Using cached fastapi-0.115.11-py3-none-any.whl.metadata (27 kB)
  Using cached importlib_metadata-8

In [31]:
pip install boto3

  Using cached boto3-1.37.17-py3-none-any.whl.metadata (6.7 kB)
  Using cached botocore-1.37.17-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.11.4-py3-none-any.whl.metadata (1.7 kB)
Using cached boto3-1.37.17-py3-none-any.whl (139 kB)
Using cached botocore-1.37.17-py3-none-any.whl (13.4 MB)
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached s3transfer-0.11.4-py3-none-any.whl (84 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /usr/local/python/3.12.1/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
import boto3
print(boto3.__version__)

1.37.17


In [5]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green taxi duration")

2025/03/20 21:21:34 INFO mlflow.tracking.fluent: Experiment with name 'green taxi duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-models-slv/1', creation_time=1742505694391, experiment_id='1', last_update_time=1742505694391, lifecycle_stage='active', name='green taxi duration', tags={}>

In [19]:
def read_dataframe(filename: str):
    df=pd.read_parquet(filename)
    df['duration']=df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration=df.duration.apply(lambda m : m.total_seconds() / 60)
    
    df=df[(df.duration >= 1) & (df.duration <= 60)]
    categorical=['PULocationID','DOLocationID']
    df[categorical]=df[categorical].astype(str)
    return df    
    

In [23]:
def prepare_dictionaries(df:pd.DataFrame):
    df['PU_DO']=df['PULocationID'] + '_' + df['DOLocationID']
    categorical=['PU_DO']
    numerical=['trip_distance']
    dicts=df[categorical + numerical].to_dict(orient='records')
    return dicts

In [24]:
df_train=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
target='duration'
y_train=df_train[target].values
y_val=df_val[target].values

dict_train=prepare_dictionaries(df_train)
dict_val=prepare_dictionaries(df_val)

In [25]:
len(df_train),len(df_val)

(73908, 61921)

In [33]:
with mlflow.start_run():
    params=dict(max_depth=20,n_estimators=100,min_samples_leaf=10,random_state=0)
    mlflow.log_params(params)

    dv = DictVectorizer()
    model = RandomForestRegressor(**params,n_jobs=-1)

    X_train = dv.fit_transform(dict_train)
    model.fit(X_train,y_train)

    X_val = dv.transform(dict_val)
    y_pred = model.predict(X_val)

   
    mse = mean_squared_error(y_val, y_pred)
    print(params,mse)
    mlflow.log_metric('mse',mse)
    
    mlflow.sklearn.log_model(pipeline,artifact_path="model")

    with open('dict_vectorizer.bin','wb') as f_out:
        pickle.dump(dv,f_out)
    

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 45.64114429815588


2025/03/20 21:51:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run bittersweet-cow-58 at: http://127.0.0.1:5000/#/experiments/1/runs/2442700890634d85896b9c7e14e73c06
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


NoCredentialsError: Unable to locate credentials

In [17]:
with mlflow.start_run():
    params=dict(max_depth=20,n_estimators=100,min_samples_leaf=10,random_state=0)
    mlflow.log_params(params)
    pipeline=make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params,n_jobs=-1)
    )
    pipeline.fit(dict_train,y_train)
    y_pred=pipeline.predict(dict_val)
    
    rmse=mean_squared_error(y_pred,y_val,squared=False)
    print(params,rmse)
    mlflow.log_metric('rmse',rmse)
    mlflow.sklearn.log_model(pipeline,artifact_path="model")
    
    

🏃 View run brawny-cow-490 at: http://127.0.0.1:5000/#/experiments/1/runs/ab4a138396224ea799e885d20955212f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


TypeError: 'NoneType' object is not iterable

In [6]:
from mlflow.tracking import MlflowClient

ModuleNotFoundError: No module named 'mlflow'

In [ ]:
MLFLOW_TRACKING_URI = ''
RUN_ID = ''
client=MlflowClient(tracking_uri= MLFLOW_TRACKING_URI)

In [ ]:
path=client.download_artifacts(run_id=RUN_ID,path='dict_vectorizer.bin')

In [ ]:
with open(path,'rb')as f_out:
    dv=pickle.load(f_out)
          